## understand automatic differentiation 

微分求解分为4类：
1. 手动求解 （manual differentiation） ：
    计算梯度公式，代入实际数值；
2. 数值微分（Numerical differentiation）：
    导数定义 (f(x+d)-f(x))/d  d->0, 但是比较慢(计算两次函数值)，且有截断误差roundoff （比如sin（x）泰勒展开，只计算前几项，后面截断）和舍入误差truncation error（计算不精确）； 一般可以用来验证梯度实现的正确性；
3. 符号微分（symbolic differentiation）：
    对数学表达式自动微分求解，但是会有表达式膨胀的问题，比如 16a(1-a)((1-2a)^2) 对a 求偏导: 16(1-a)((1-2a)^2)-16a((1-2a)^2)-64a(1-a)(1-2a)
4. 自动微分 （Automatic differentiation）：
    自动微分法是一种介于符号微分和数值微分的方法：数值微分强调一开始直接代入数值近似求解；符号微分强调直接对代数进行求解，最后才代入问题数值；自动微分将符号微分法应用于最基本的算子，比如常数，幂函数，指数函数，对数函数，三角函数等，然后代入数值，保留中间结果，最后再应用于整个函数。因此它应用相当灵活，可以做到完全向用户隐藏微分求解过程，由于它只对基本函数或常数运用符号微分法则，所以它可以灵活结合编程语言的循环结构，条件结构等，使用自动微分和不使用自动微分对代码总体改动非常小，并且由于它的计算实际是一种图计算，可以对其做很多优化. 分为forwoard mode 和reverse mode。
    

# forward mode

前向模式在计算图前向传播时同时计算微分
$ v_i=\frac{delta(v_i)}$

In [ ]:


        
class Op(object):
    def compute(self, nodes):
        assert False, "Implemented in subclass"
    def gradient(self, nodes)
        assert False, "Implemented in subclass"
        
# 实际计算在各个操作算子上        
class MulOp(Op):
    def compute(self, node1, node2):
        return node1.real * node2.real
    def gradient(self, node1, node2):
        return node1.real*node2.grad
        
        
        
'''
a = 1 + i
b = 2 + j
a*b = (2 + 2i + 1j + 2ij) 
ij=0 // not real value
grad(a) = 2
grad(b) = 1
'''
def dot_product(a_num, b_num):
    real = np.dot(a_num.real, b_num.real)
    jnum = np.dot(a_num.real, b_num.jnum)
    jnum += np.dot(b_num.real, a_num.jnum)
    return DualNum(real, jnum)


    
        
    

# backward mode

In [81]:
import numpy as np
import math

class Node(object):
    def __init__(self,inputs,grads=1):
        self.val=inputs
        self.grad=grads

class Op(object):
    def __init__(self,name):
        self._name=name
        self.node=Node(name)
        self.inputs=[]
        self.node.grads=0
    def compute(self, nodes):
        assert False, "Implemented in subclass"
    def gradient(self, nodes):
        assert False, "Implemented in subclass"
        

# 实际计算在各个操作算子上   
class Const(Op):
    def compute(self, val):
        self.node.val=val
        self.inputs=[]
        return val
    def gradient(self,out_grad):
        grads=out_grad
        self.node.grads=grads
        return grads

       
class MulOp(Op):
    def compute(self, node1, node2):
        inputs= node1.val * node2.val
        self.inputs=[node1,node2]
        self.node.val=inputs
    
    def gradient(self, out_grad):
        self.node.grads=out_grad
        grads=[self.inputs[1].val * out_grad ,self.inputs[0].val*out_grad]
        self.inputs[0].grads+=grads[0]
        self.inputs[1].grads+=grads[1]
        
class AddOp(Op):
    def compute(self, node1, node2):
        inputs = node1.val+ node2.val
        self.inputs=[node1,node2]
        self.node.val=inputs
        
    def gradient(self, out_grad):
        self.inputs[0].grads+=out_grad
        self.inputs[1].grads+=out_grad
        
        
class SubOp(Op):
    def compute(self, node1, node2):
        inputs = node1.val-node2.val
        self.inputs=[node1,node2]
        self.node.val=inputs
        
    def gradient(self, out_grad):
        self.inputs[0].grads+=out_grad
        self.inputs[1].grads-=out_grad

class In(Op):
    def compute(self, node1):
        inputs=math.log(node1.val,math.e)
        self.inputs=[node1]
        self.node.val=inputs
    
    def gradient(self, out_grad):
        self.inputs[0].grads+=out_grad*(1/self.inputs[0].val)
    
class Sin(Op):
    def compute(self, node):
        inputs=np.sin(node.val)
        self.inputs=[node]
        self.node.val=inputs
    def gradient(self, out_grad):
        self.inputs[0].grads+=out_grad*np.cos(self.inputs[0].val)
    
    
node1=Const("const1")
node1.compute(2)
node2=Const("const2")
node2.compute(5)

in_op=In("in")
in_op.compute(node1.node)
mul_op=MulOp("mul1")
mul_op.compute(node1.node,node2.node)
sin_op=Sin("sin")
sin_op.compute(node2.node)
add_op=AddOp("add")
add_op.compute(in_op.node,mul_op.node)
sub_op=SubOp("sub")
sub_op.compute(add_op.node,sin_op.node)
print("forward result:",sub_op.node.val)

sub_op.gradient(1)
add_op.gradient(sub_op.inputs[0].grads)
in_op.gradient(add_op.inputs[0].grads)
mul_op.gradient(add_op.inputs[1].grads)
sin_op.gradient(sub_op.inputs[1].grads)
print("the differential of final result to node1: ", node1.node.grads)
print("the differential of final result to node2: ", node2.node.grads)


forward result: 11.652071455223084
the differential of final result to node1:  5.5
the differential of final result to node2:  1.7163378145367738
